In [ ]:
# Auto-Reload Setup für Entwicklung
# Automatisches Neuladen aller Module bei Änderungen
%load_ext autoreload
%autoreload 2

# Persönliche Dokumentation für das Tool
## Anforderungen
- CSV Exportdatei aus mLife entgegen nehmen ("gesamte_akte.csv")
- Alle (oder zumindest für RedCap erforderliche) Daten in ein Dataframe parsen
- Darstellung und Konfiguration des Exports mit Streamlit
- Export als CSV, welche in RedCap importiert werden kann

## Übersicht Datenfluss
CSV-Import 
-> Parsing 
-> Dataframe für jede Kategorie 
-> Auswahl in Streamlit & Aggregation
-> RedCap-DB optimiertes DataFrame
-> CSV-Export

## Import
- aus mLife wird eine CSV-Datei der gesamten_akte exportiert; der Delimiter kann zwischen ";" und "|" gewählt werden und wird automatisch erkannt
### Parsing der Daten in den State (`StateProvider`)
- `state_provider.state_provider_class StateProvider` initiiert sich selbst und exportiert sich als `state_provider`
    - `state_provider` verwaltet und liefert App-weit alle Daten in Form der Pydantic-Klasse `schemas.app_state_schema.app_state AppState`
        - `ParsedData`, `Views` und `UiState`
- `state_provider.parse_data_to_state()` nimmt die importierte CSV-Datei entgegen
    - übergibt die Datei an den Parser `services.data_parser DataParser`, wo durch den Aufruf `DataParser(file, delimiter)` die Parser-Klasser initiiert wird
    - StateProvider ruft die einzelnen Parser-Methoden auf, die wiederum Dataframes für jede Kategorie an Daten (bspw. Laborwerte, Vitalwerte, NIRS) liefern
    - StateProvider erzeugt aus allen generierten DataFrames eine Instanz der Pydantic-Klasse `schemas.app_state_schemas.app_state ParsedData`
    - StateProvider legt diese `ParsedData`-Instanz im `AppState.parsed_data`ab
### Verarbeitung im `DataParser`
- Bei Initiierung lädt der DataParser erstmal nur das File
- wenn das erste mal eine Parsing-Funktion aufgerufen wird, wird die Datei mit `DataParser._split_blocks()` und `DataParser._clean_csv()` bereinigt und als Objekt mit einzelnen Datenblöcken gespeichert nach dem Schema in `DataParser.header`
- `DataParser`enthält dann verschiedene Parser-Methoden, die aus diesem `blocks`-Objekt Pandas DataFrames anhand der Pydantic-Schemata aus `schemas.parse_schemas` erzeugt
    - `_parse_table_data`
    - `parse_nirs_logic()`
    - `parse_medication_logic()`
    - `parse_fluidbalance_logic()`
    - `parse_from_all_patient_data()`
    - `parse_all_patient_data()` (wird von o.g. Funktionen benötigt, allerdings auch externer Aufruf mgl.)
    - `parse_respiratory_data()`
### Datenimport im UI
#### Homepage
- Hier wird die gesamte_akte.csv hochgeladen und der Delimiter festegelegt
- Wird nur angezeigt, wenn `AppState.parsed_data`
#### Sidebar
- Gibt Zugriff auf die verschiedenen Views und den ausgewählten Zeitraum für die Aggregation und Export-Logik
#### Overview
- Gesamtzahl der parsed Datensätze wird angezeigt
- Zeitraum der Aufzeichnungen wird angezeigt mit `state_provider.get_time_range()`
- Zeiträume für einzelne Geräte (Impella, ECMO) wird angezeigt mit `state_provider.get_device_time_ranges()`



## Aggregation
- Für die Abfrage von Daten ist ebenfalls der `state_provider` zuständig

In [2]:
# Alle Methoden von state_provider
from state_provider.state_provider_class import state_provider
def show_available_methods():
    methods = [method for method in dir(state_provider) if not method.startswith('_')]
    print("Verfügbare StateProvider Methoden:")
    for i, method in enumerate(methods, 1):
        print(f"  {i:2}. {method}")
    return methods

# Methoden anzeigen
available_methods = show_available_methods()

Verfügbare StateProvider Methoden:
   1. data_parser
   2. get_device_time_ranges
   3. get_respiration_type
   4. get_respiratory_value
   5. get_selected_view
   6. get_state
   7. get_time_of_mcs
   8. get_time_range
   9. get_vasoactive_agents_df
  10. get_vitals_value
  11. has_device_past_24h
  12. has_mcs_records_past_24h
  13. has_parsed_data
  14. parse_data_to_state
  15. query_data
  16. reset_state
  17. save_state
  18. set_selected_time_range
  19. update_state


Die mächtigste Methode ist hier `StateProvider.query_data()`
- Neben einfachen Abfragen von ganzen Kategorien ermöglicht sie queries mit sehr differenzierten abfage-Strings
- mögliche Filter für `value_strategy`: "first", "last", "median", "mean", "nearest"
- im Falle von `"nearest"` wird noch die property `"nearest_time"` erforderlich, welche `datetime.date()` erwartet

In [ ]:
from datetime import datetime, time
from services.delimiter_auto_detecion import detect_delimiter

file = "data/gesamte_akte2.csv"
delimiter = detect_delimiter(file)
state = state_provider.parse_data_to_state(file, delimiter)

vitals = state_provider.query_data("vitals")
ecmo = state_provider.query_data("ecmo")

test_date = datetime(2025,9,15).date()
art = state_provider.query_data("vitals", {'timestamp': test_date, 'parameter': "ARTs", 'value_strategy': "median"})

heparin = state_provider.query_data("medication", {"medication": "heparin", "stop": test_date})

near_lactat = state_provider.query_data("lab", {"timestamp": datetime(2025,9,15).date(), "parameter": "Lactat","category": "Blutgase arteriell", "value_strategy": "nearest", "nearest_time": time(12, 0)})


### Data Exploration (UI)
- Views für alle Kategorien werden in der `sidebar` zugänglich gemacht
- View ruft alle verfügbaren Daten der Kategorie mit `StateProver.query_data()` ab
- View erzeugt eine Auswahl von Unter-Kategorie, Parametern, Zeitraum, Value-Strategy => `StateProver.query_data()`
- View zeigt die nach dieser Auswahl gefilterten Daten als Tabelle(n)
- Views zeigt die nach dieser Auswahl gefilterten numerischen Daten graphisch

### RedCap Form (UI) - Exemplarisch für LabModel
- Voraussetzung ist neben dem Datensatz die Auswahl des Zeitfensters (`AppState.selected_time_range` wird default auf ECMO/Impella Start & - Ende gesetzt) und einer Record_id (`AppState.record_id` via `StateProvider.update_state()`)
    - MCS-Zeiträume werden mit bspw. `StateProvider.get_device_time_ranges("impella")` ermittelt (Liste iterierbarer dataclass mit `device`, `start` und `end`)
- Basierend auf `AppState.selected_time_range` wird für jeden Tag `schemas.db_schemas.lab LabModel` mithilfe des `StateProvider.query_data()` befüllt
    - `LabModel.redcap_event_name` Ist je nach Datensatz baseline oder impella_arm_2 oder ecls_arm_2 und muss mit übergeben werden
        - einige repat_instruments werden baseline zugewiesen (bspw. koro-befunde)
        - Messreihen der Impella (Einstellungen wie P-Level) werden impella_arm_2 zugewiesen, Messreihen der ECMO entspr. ECLS
        - Laborwerte werden doppelt hinterlegt - einmal für jeden Arm. Dabei sind die Werte nicht zwangsläufig identisch, sondern vom 24h-Intervall der Implantation des entspr. Device abhängig (bei Routine-Labor idR identisch da 1x tgl, bei BGA idR verschieden da zeitlich verschoben)
    - `LabModel.redcap_repeat_instrument` wird automatisch belegt (bei Labor bspw. mit "Lab")
    - `LabModel.redcap_repeat_instance` zählt mit jedem Tag hoch
- Uhrzeit wird bei `value_strategy = "nearest"` anhand von `AppState.nearest_time` festgelegt und kann hier festegelegt werden mit 2 Pickern
    - (als Vorschlag werden die Implantationszeiten von ECMO & Impella voreingestellt)
- dieses LabModel wird dann in `AppState.lab_form : list[LabModel]` gespeichert

In [ ]:
from services.value_aggregation.lab_aggregator import LabAggregator
from datetime import date

state_provider.update_state(record_id="123wqewer")

lab_aggregator = LabAggregator(
    state_provider, 
    date= date(2025, 9, 19), 
    record_id=state_provider.get_record_id(), 
    redcap_event_name="impella_arm_2", 
    redcap_repeat_instrument="labor", 
    redcap_repeat_instance=1, 
    value_strategy="nearest", 
    nearest_time=state_provider.get_nearest_impella_time())


#### Data Aggregation / Export Builder Flow
- Über die `sidebar` wird im Expander "Redcap" der `views.export_builder` aufgerufen
    - hier werden Zeiträume und Zeitpunkte nochmal konfiguriert
    - Wenn keine Record-ID vorhanden ist, muss diese spätestens hier auch eingegeben werden
- Wenn die Export-Formulare im State Daten enthalten (bspw. `AppState.lab_form`), also eine Aggregation erfolgt ist, werden in diesem Expander auch die entsprechenden Views angeboten
